In [1]:
import osmnx as ox
import pandas as pd
from routing import astar_route
import folium
import matplotlib.pyplot as plt
import routing_utils as ru


In [2]:
G = ru.load_graph() # Load Graph can take 2-5 minutes, due to large graph size

In [3]:
origin = (33.974801, -118.279684)
dest = (34.040984, -118.288077)
weather = "Clear"

ru.VisualizeMap(G, origin, dest, weather)

[123280905, 1894071804, 123280903, 123280900, 122585932, 123280897, 123113181, 122753225, 123280890, 123272400, 122949018, 4020269297, 1720047785, 1720047803, 122994037, 122994036, 123077382, 123275890, 122865585, 123672794, 123656161, 123368411, 122572354, 123487839, 123101239, 122742725, 123295469, 122853187, 122853193, 123672789, 123672786, 123672783, 122745702, 123205339, 122840848, 122855953, 123672780, 123358502, 1732243724, 11333380439, 1732243606, 1732243671, 1732243934, 216708772, 123192467, 216708788, 123451079, 1732243507, 269634616, 1732243532, 269634707, 122609799, 1732243679, 122670273, 123571748, 1732243590, 1732243580, 21302782, 269633021, 6815182444, 123280934, 123292045, 123292100, 1841835270, 63068643, 1286136422, 9747602369, 21306060, 5237381975, 122719216, 6813405266, 122814447, 6816193698, 6816193696, 123408740, 6816180168, 6816180170, 1843466234, 122674590, 122740204, 123161914, 123161946, 212387023, 122918143, 14940433, 122770966, 1840669350, 14940434, 123454932

In [ ]:
##########################################################################################################################################################################################

In [ ]:
def path_finding(G, origin, dest, use_risk):
    G = ru.get_edge_data(G)
    cost_attr = "cost_risk" if use_risk else "cost_distance"

    #origin = (34.06655181863482, -118.18789017013762)
    #dest = (34.0647417476385, -118.25823027030046)

    if use_risk:
        node_path, edge_path, road_names = astar_route(
            G,
            origin_point=origin,
            destination_point=dest,
            weight=cost_attr
        )
        print(node_path)
        print(f"Number of nodes in path: {len(node_path)}")
        print(f"Number of edges in path: {len(edge_path)}")
        print("Roads to follow:", road_names)

    # Compute fastest route (minimize travel time)
    node_path_fast, edge_path_fast, road_names_fast = astar_route(
        G,
        origin_point=origin,
        destination_point=dest,
        weight="cost_length",
    )
    
    print(node_path_fast)
    print(f"Number of nodes in path (fastest route): {len(node_path_fast)}")
    print(f"Number of edges in path (fastest route): {len(node_path_fast)}")
    print("Roads to follow(fastest route): ", road_names_fast)

    return G, edge_path, edge_path_fast

In [ ]:
def VisualizesMapOld(G, origin, dest):
    G, edge_path, edge_path_fast = ru.path_finding(G, origin, dest)

    # Calculate total travel time for each route (in minutes)
    total_time_risk = sum(G.edges[u, v, key]["cost_time"] for u, v, key in edge_path) * 60
    total_time_fast = sum(G.edges[u, v, key]["cost_time"] for u, v, key in edge_path_fast) * 60

    # Create a folium map centered on the origin
    m = folium.Map(location=origin, zoom_start=13)
    counter = -1
    
     ## Risk Route Coloring ###
    edge_path = ru.path_finding(G)[2]
    for u, v, key in edge_path:
        counter+=1
        data = G.edges[u, v, key]
        if 'geometry' in data:
            line = data['geometry']
            coords = [(lat, lon) for lon, lat in line.coords]
            if counter == 0:
                first_node_coords = coords[0]

        else:
            point_u = (G.nodes[u]['y'], G.nodes[u]['x'])
            point_v = (G.nodes[v]['y'], G.nodes[v]['x'])
            #line = LineString([point_u, point_v])
            coords = [point_u, point_v]
        length = data.get("length", 1.0)
        cost_risk = data.get("cost_risk", length)
        base_risk = (cost_risk / length) - 1
        color = ru.risk_to_color(base_risk)
        folium.PolyLine(
            locations=coords,#[(lat, lon) for lon, lat in line.coords],
            color=color,
            weight=5,
            opacity=0.8,
            popup=f"{data.get('name', 'Unnamed Road')}<br>Risk: {base_risk:.2f}"
        ).add_to(m)
        # Add the start coords to first node line and end coords to last node line
    folium.PolyLine(locations=[origin, first_node_coords],
                    color=first_color,
                    weight=5,
                    opacity=0.8,
                    popup=f"{data.get('name', 'Unnamed Road')}<br>Risk: {base_risk:.2f}"
                    ).add_to(m)
    print('edge_path_end:', edge_path[-1])
    last_u, last_v, last_key = edge_path[-1]
    data = G.edges[last_u, last_v, last_key]

    if 'geometry' in data:
        last_coords = (data['geometry'].coords[-1][1], data['geometry'].coords[-1][0])  # (lat, lon)
    else:
        last_coords = (G.nodes[last_v]['y'], G.nodes[last_v]['x'])  # fallback if no geometry


    edge_path_fast = ru.path_finding(G)[2]
    ### Fastest Route Coloring ###
    for u, v, key in edge_path_fast:
        data = G.edges[u, v, key]
        if 'geometry' in data:
            line = data['geometry']
            coords = [(lat, lon) for lon, lat in line.coords]
            
        else:
            point_u = (G.nodes[u]['y'], G.nodes[u]['x'])
            point_v = (G.nodes[v]['y'], G.nodes[v]['x'])
            #line = LineString([point_u, point_v])
            coords = [point_u, point_v]
        folium.PolyLine(
            locations=coords,#[(lat, lon) for lon, lat in line.coords],
            color="blue",
            weight=3,
            opacity=0.7,
            popup=f"Fastest: {data.get('name', 'Unnamed Road')}"
        ).add_to(m)

    

    folium.PolyLine(locations=[last_coords, dest],
                    color=last_color,
                    weight=5,
                    opacity=0.8,
                    popup=f"{data.get('name', 'Unnamed Road')}<br>Risk: {base_risk:.2f}"
                    ).add_to(m)
    # Add start and end markers
    folium.Marker(origin, tooltip="Start", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker(dest, tooltip="Destination", icon=folium.Icon(color="red")).add_to(m)

    # Add info box with travel times
    info_html = f"""
    <div style="position: fixed; 
                top: 10px; left: 60px; width: 250px; height: 100px; 
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; padding: 10px;">
    <b>Route Time Estimates</b><br>
    <span style='color:black;'>Safest route: </span>{total_time_risk:.1f} min<br>
    <span style='color:blue;'>Fastest route: </span>{total_time_fast:.1f} min
    </div>
    """
    m.get_root().html.add_child(folium.Element(info_html))

    # Risk legend
    legend_html = """
    <div style="position: fixed; 
                top: 10px; left: 280px; width: 150px; height: 100px; 
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; padding: 10px;">
    <b>Risk Level</b><br>
    <span style="color:#00ff00;">●</span> Low<br>
    <span style="color:#ffff00;">●</span> Medium<br>
    <span style="color:#ff0000;">●</span> High
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    # Map styles
    folium.TileLayer(titles = "Stamen Terrain").add_to(m)
    folium.TileLayer("CartoDB Positron").add_to(m)
    folium.TileLayer("openstreetmap").add_to(m)
    folium.LayerControl().add_to(m)

    # Save to HTML
    m.save("risky_path.html")
    print("Map saved to risky_path.html")


In [ ]:
origin = (33.974801, -118.279684)
dest = (34.010984, -117.276077)

VisualizesMapOld(G, origin, dest, use_risk=True)

TypeError: VisualizesMap() got an unexpected keyword argument 'use_risk'